In [1]:
import os
import random
import skimage.data
from skimage import io
import skimage.transform as skt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [2]:
# submap_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhaun_448_test_with_no_barrier1/submap"
# pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhaun_448_test_with_no_barrier1/picture"

# # for i in range(4):
# i = 1
# submap1 = cv2.imread(os.path.join(submap_dir,str(1189) + '.pgm'))
# submap1 = cv2.resize(submap1,(448,448))
# submap1 = cv2.transpose(submap1)
# submap1 = cv2.flip(submap1, 1)
# # submap1 = np.rot90(submap1)
# cv2.imshow("submap",submap1)


# pic1 = cv2.imread(os.path.join(pic_dir, '01189.jpg'))
# cv2.imshow("pic",pic1)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(np.shape(submap1))

In [3]:
train_label_dir = "/home/ysk/database/test_data/benhaun_test"
train_pic_dir = "/home/ysk/database/test_data/benhaun_test/picture"
train_submap_dir = "/home/ysk/database/test_data/benhaun_test/submap"

# train_label_dir = "/home/ysk/database/train_data_map_pic/benhuan_train"
# train_pic_dir = "/home/ysk/database/train_data_map_pic/benhuan_train/picture"
# train_submap_dir = "/home/ysk/database/train_data_map_pic/benhuan_train/submap"

train_txt = os.path.join(train_label_dir,"label.txt")
count = len(open(train_txt,'r').readlines()) - 1   #1190
file = open(train_txt,'r')
s = file.readlines()

def get_train_data_batch(i, pic_dir=train_pic_dir, label_dir=train_label_dir,submap_dir=train_submap_dir):
    labels = []
    images = []
    
    image1 = skimage.data.imread(os.path.join(pic_dir, s[i].split(";")[0]))
    submap1 = cv2.imread(os.path.join(submap_dir,str(i) + '.pgm'))
    submap1 = cv2.resize(submap1,(448,448))
    submap1 = cv2.transpose(submap1)
    submap1 = cv2.flip(submap1, 1)
    
#     image2 = skimage.data.imread(os.path.join(pic_dir, s[i+1].split(";")[0]))
#     submap2 = cv2.imread(os.path.join(submap_dir,str(i+1) + '.pgm'))
#     submap2 = cv2.resize(submap2,(448,448))
#     submap2 = cv2.transpose(submap2)
#     submap2 = cv2.flip(submap2, 1)
    
    
#     img = cv2.merge([image1,submap1,image2,submap2])
    img = cv2.merge([image1,submap1])
    images.append(img)
    
    label = s[i].split(";")[1:]
    label = list(map(float,label))
    labels.append(label)
    
    return np.asarray(images),np.asarray(labels)
    
num = 0
train_images, train_labels = get_train_data_batch(num)
print(np.shape(train_images))
print(np.shape(train_labels))
print(train_labels)

(1, 448, 448, 6)
(1, 3)
[[ 0.25599488 -0.06013947 -6.1226604 ]]


In [4]:
with tf.name_scope("input"):
    input_image = tf.placeholder("float", [None, 448,448,6],name='input_image')/255.*2.-1
    y_ = tf.placeholder("float", [None, 3],name='y_') 

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


#第0层卷积
with tf.name_scope("conv_0"):
    W_conv0 = weight_variable([7, 7, 6, 64])
    b_conv0 = bias_variable([64])
    conv_layer0 = tf.nn.conv2d(input_image, W_conv0, strides=[1,1,1,1], padding="SAME") + b_conv0
    conv_layer_nor0 = tf.layers.batch_normalization(conv_layer0,training=False)
    
    h_conv0 = tf.nn.leaky_relu(conv_layer_nor0)
with tf.name_scope("max_pool_0"):
    h_pool0 = max_pool_2x2(h_conv0)   #输出变成224*224*64 [None,224,224,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度
with tf.name_scope("conv_1"):
    W_conv1 = weight_variable([5, 5, 64, 128])
    b_conv1 = bias_variable([128])
    conv_layer1 = conv2d(h_pool0, W_conv1) + b_conv1
    conv_layer_nor1 = tf.layers.batch_normalization(conv_layer1,training=False)

    h_conv1 = tf.nn.leaky_relu(conv_layer_nor1)
# h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1, strides=[1,2,2,1], padding="SAME") + b_conv1)
with tf.name_scope("max_pool1"):
    h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64   [None,112,112,64]

#第二层卷积层
with tf.name_scope("conv_2"):
    W_conv2 = weight_variable([5, 5, 128, 192])
    b_conv2 = bias_variable([192])
    conv_layer2 = conv2d(h_pool1, W_conv2) + b_conv2
    conv_layer_nor2 = tf.layers.batch_normalization(conv_layer2,training=False)

    h_conv2 = tf.nn.leaky_relu(conv_layer_nor2)
with tf.name_scope("max_pool_2"):
    h_pool2 = max_pool_2x2(h_conv2)  #输出变为56*56*192


#第三层卷积层
with tf.name_scope("conv_3"):
    W_conv3 = weight_variable([3, 3, 192, 128])
    b_conv3 = bias_variable([128])
    conv_layer3 = conv2d(h_pool2, W_conv3) + b_conv3
    conv_layer_nor3 = tf.layers.batch_normalization(conv_layer3,training=False)
    

    h_conv3 = tf.nn.leaky_relu(conv_layer_nor3)
with tf.name_scope("max_pool_3"):
    h_pool3 = max_pool_2x2(h_conv3)   #输出为28*28*128


#第四层卷积层
with tf.name_scope("conv_4"):
    W_conv4 = weight_variable([3, 3, 128, 64])
    b_conv4 = bias_variable([64])
    conv_layer4 = conv2d(h_pool3, W_conv4) + b_conv4
    conv_layer_nor4 = tf.layers.batch_normalization(conv_layer4,training=False)

    h_conv4 = tf.nn.leaky_relu(conv_layer_nor4)
with tf.name_scope("max_pool_4"):
    h_pool4 = max_pool_2x2(h_conv4)   #输出变为14*14*64


keep_prob = tf.placeholder("float")

#密集连接层0
with tf.name_scope("fc0"):
    W_fc0 = weight_variable([14*14*64, 2048])
    b_fc0 = bias_variable([2048])

    h_pool4_flat = tf.reshape(h_pool4, [-1, 14*14*64])
    
    out_fc0 = tf.matmul(h_pool4_flat, W_fc0) + b_fc0
    out_nor_fc0 = tf.layers.batch_normalization(out_fc0,training=False)
    
    h_fc0 = tf.nn.leaky_relu(out_nor_fc0)   #输出为96*2048

    #relu函数作用是求出max(h_fc1,0)
    h_fc1_drop0 = tf.layers.dropout(h_fc0, keep_prob)   #输出为96*2048

#密集连接层1
with tf.name_scope("fc1"):
    W_fc1 = weight_variable([2048, 1024])
    b_fc1 = bias_variable([1024])
    
    out_fc1 = tf.matmul(h_fc1_drop0, W_fc1) + b_fc1
    out_nor_fc1 = tf.layers.batch_normalization(out_fc1,training=False)

    h_fc1 = tf.nn.leaky_relu(out_nor_fc1)   #输出为2048*1024

    #relu函数作用是求出max(h_fc1,0)
    h_fc1_drop1 = tf.layers.dropout(h_fc1, keep_prob)   #输出为2048*1024

#密集连接层2
with tf.name_scope("fc2"):
    W_fc2 = weight_variable([1024, 512])
    b_fc2 = bias_variable([512])
    
    out_fc2 = tf.matmul(h_fc1_drop1, W_fc2) + b_fc2
    out_nor_fc2 = tf.layers.batch_normalization(out_fc2,training=False)

    h_fc2 = tf.nn.leaky_relu(out_nor_fc2)
    #relu函数作用是求出max(h_fc1,0)
    h_fc1_drop2 = tf.layers.dropout(h_fc2, keep_prob)
    
#输出层
with tf.name_scope("out_layer"):
    W_fc3 = weight_variable([512, 3])
    y_conv = tf.matmul(h_fc1_drop2, W_fc3)

    
delta=tf.constant(1.25)
#huber损失函数
loss1 = tf.reduce_mean(tf.multiply(tf.square(delta),
                                          tf.sqrt(1. + tf.square((y_[:2] - y_conv[:2])/delta)) - 1.))
loss2 = 500 * tf.reduce_mean(tf.multiply(tf.square(delta),
                                          tf.sqrt(1. + tf.square((y_[2:] - y_conv[2:])/delta)) - 1.))
loss = loss1 + loss2


checkpointsPath = "/home/ysk/database/train_data/benhuan/fc_bn_checkpoints_4805/"
path = os.getcwd()
test_result_path = os.path.join(path,'test_result','new_test_6in_3out_test_result.txt')

if os.path.exists(test_result_path):
    os.remove(test_result_path)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
    if checkPoint and checkPoint.model_checkpoint_path:
        saver.restore(sess,checkPoint.model_checkpoint_path)
        print("restored %s" % checkPoint.model_checkpoint_path)
        
    else:
        print("no checkpoint found!")
        
    for i in range(count-2):
        batch_x,batch_y = get_train_data_batch(i)
        batch_y = batch_y*10000
        
        y_pre = sess.run(y_conv,feed_dict={input_image:batch_x, y_:batch_y,keep_prob:1.0})
              
        with open(test_result_path,'a') as f:
            for i in range(3):
                f.write(str(batch_y[0][i]) + ',')
            for i in range(2):
                f.write(str(y_pre[0][i]) + ',')
            f.write(str(y_pre[0][2]) + '\n')

        print("real location is:\n",batch_y)
        print("prediction is:\n",y_pre)
        print("\n")
    print("done!")


INFO:tensorflow:Restoring parameters from /home/ysk/database/train_data/benhuan/fc_bn_checkpoints_4805/saved_net.ckpt-199000
restored /home/ysk/database/train_data/benhuan/fc_bn_checkpoints_4805/saved_net.ckpt-199000
real location is:
 [[  2559.94878646   -601.39466037 -61226.6039933 ]]
prediction is:
 [[  6611.1445   2414.3284 -52533.906 ]]


real location is:
 [[  3628.3898691    -653.78329356 -46311.8708672 ]]
prediction is:
 [[ 19674.861    1927.4335 -41299.11  ]]


real location is:
 [[  4660.80391257   -713.05870336 -37682.2471485 ]]
prediction is:
 [[29667.043 12034.835 -9769.773]]


real location is:
 [[  5694.51729164   -757.93697074 -39235.9535478 ]]
prediction is:
 [[ 14588.324  12846.366 -51560.867]]


real location is:
 [[  6756.30097055   -848.62083795 -38530.0963466 ]]
prediction is:
 [[ 27211.357   5995.176 -34636.13 ]]


real location is:
 [[ 7807.6845113   -882.19755469 -7146.68964555]]
prediction is:
 [[20525.09  11051.607 24781.258]]


real location is:
 [[8860.0682

real location is:
 [[77583.453617    -109.40832222  7171.68709474]]
prediction is:
 [[-165093.95  193153.4  -693404.06]]


real location is:
 [[78567.4297328   -121.81863826 10637.2727279 ]]
prediction is:
 [[-163818.48  166690.2  -675407.9 ]]


real location is:
 [[79609.9814589   -125.24339174  8467.68035461]]
prediction is:
 [[-172841.72  198952.14 -778293.  ]]


real location is:
 [[80731.9325902   -128.90559939  9782.66189967]]
prediction is:
 [[-160115.75  142695.73 -605906.2 ]]


real location is:
 [[81779.1080814   -124.92300889 17480.436746  ]]
prediction is:
 [[-172542.23  170391.5  -800375.44]]


real location is:
 [[82854.8385365  -102.3178427 25000.9458922]]
prediction is:
 [[-177770.34  203090.94 -971768.  ]]


real location is:
 [[ 8.39197939e+04 -7.70294905e+01  2.52418084e+04]]
prediction is:
 [[-173099.28  168398.98 -844734.25]]


real location is:
 [[ 8.49462426e+04 -6.06783679e+01  3.00066376e+04]]
prediction is:
 [[-116927.51   83780.9  -459551.  ]]


real location

real location is:
 [[149743.953322   43977.2466914 453045.94898  ]]
prediction is:
 [[  29971.209  113809.45  -205729.36 ]]


real location is:
 [[150666.422391   44878.7348914 452931.271256 ]]
prediction is:
 [[  32101.527  110326.57  -157299.27 ]]


real location is:
 [[151585.389729   45768.4624687 451675.563026 ]]
prediction is:
 [[  15549.269  110393.6   -372458.66 ]]


real location is:
 [[152481.807829   46645.6000256 451775.687992 ]]
prediction is:
 [[  47282.734   99492.54  -454804.   ]]


real location is:
 [[153409.111969   47550.2299475 453594.452345 ]]
prediction is:
 [[  24253.068  101283.1   -475854.12 ]]


real location is:
 [[154322.006948   48465.8926158 455887.164216 ]]
prediction is:
 [[  29931.932   91360.805 -358331.7  ]]


real location is:
 [[155274.944029   49376.4593189 452827.966926 ]]
prediction is:
 [[ 102765.445   78879.39  -518259.44 ]]


real location is:
 [[156175.244345   50221.7526377 446880.973412 ]]
prediction is:
 [[  95175.34   78197.   -232255.48

real location is:
 [[204535.362539 105597.599906 647977.748523]]
prediction is:
 [[143843.2   108971.625 433820.28 ]]


real location is:
 [[204940.727447 106552.422494 695644.09088 ]]
prediction is:
 [[112648.35 129064.53 469962.53]]


real location is:
 [[205280.958143 107556.44423  734850.037664]]
prediction is:
 [[205747.5  183830.42 855057.25]]


real location is:
 [[205552.06442  108542.255044 774802.990382]]
prediction is:
 [[101051.71 191686.1  533469.9 ]]


real location is:
 [[205752.478682 109578.660741 815438.571206]]
prediction is:
 [[120292.58 193468.   354004.1 ]]


real location is:
 [[205879.71758  110667.452864 851553.568624]]
prediction is:
 [[207666.88 145084.31 811591.3 ]]


real location is:
 [[205964.173221 111721.829933 875248.994068]]
prediction is:
 [[206593.6  147768.52 864194.4 ]]


real location is:
 [[206006.135873 112835.921275 899965.843441]]
prediction is:
 [[213123.3  150298.34 933126.94]]


real location is:
 [[206024.070538 113950.361839 914045.48894

real location is:
 [[206227.605252 183591.437202 924783.507135]]
prediction is:
 [[211818.44 179627.42 925043.1 ]]


real location is:
 [[206187.806357 184640.925785 921642.243475]]
prediction is:
 [[214824.39 195605.9  942435.06]]


real location is:
 [[206152.183372 185713.61972  917922.436958]]
prediction is:
 [[216336.11 206217.64 935326.25]]


real location is:
 [[206130.669231 186786.398331 911349.880419]]
prediction is:
 [[208856.58 193868.92 910194.94]]


real location is:
 [[206117.600047 187836.967447 910160.62935 ]]
prediction is:
 [[215998.75 218148.97 956625.6 ]]


real location is:
 [[206124.687499 188933.51705  905660.802288]]
prediction is:
 [[216844.88 227999.36 929949.6 ]]


real location is:
 [[206119.138985 190047.294682 905507.06959 ]]
prediction is:
 [[206399.47 198597.31 898841.25]]


real location is:
 [[206118.368554 191130.68332  901893.74878 ]]
prediction is:
 [[205517.14 190323.78 878697.06]]


real location is:
 [[206135.448491 192211.378086 901552.038329]]

...

real location is:
 [[ 80624.5490949 409037.457522   20492.5752664]]
prediction is:
 [[ 73439.47 388605.47 106454.03]]


real location is:
 [[ 81735.6495454 409063.972155   19920.0579578]]
prediction is:
 [[ 75782.28  393422.2   102732.836]]


real location is:
 [[ 82810.570555  409080.336348   12516.8298679]]
prediction is:
 [[ 71497.69 393173.25 116040.27]]


real location is:
 [[ 83895.8160363  409094.134082     9735.68426707]]
prediction is:
 [[ 71402.28 390942.2  121920.83]]


real location is:
 [[ 84995.9518906  409112.133376     9603.99998831]]
prediction is:
 [[ 70485.8  389915.38 126972.59]]


real location is:
 [[ 86080.4850078  409105.457551     9774.92741031]]
prediction is:
 [[ 70664.68 392909.7  121612.8 ]]


real location is:
 [[ 87145.7221315  409125.585732     9352.44004213]]
prediction is:
 [[ 75020.21 391264.38 127966.06]]


real location is:
 [[ 88236.1019797  409131.286566     9018.46359033]]
prediction is:
 [[ 85616.53 380570.72 135519.98]]


real location is:
 [[ 

real location is:
 [[ 144588.846566  368301.294348 -427704.067265]]
prediction is:
 [[ 123792.89  380136.06 -401192.78]]


real location is:
 [[ 145351.031558  367569.439225 -428327.740441]]
prediction is:
 [[ 130117.1   369643.12 -433925.47]]


real location is:
 [[ 146202.029447  366756.767056 -432714.532495]]
prediction is:
 [[ 129483.93  360822.16 -432458.2 ]]


real location is:
 [[ 147023.430035  365962.032696 -432350.564556]]
prediction is:
 [[ 127178.58  372147.1  -404091.4 ]]


real location is:
 [[ 147879.790737  365147.686044 -433279.607867]]
prediction is:
 [[ 134527.22  358341.56 -442130.25]]


real location is:
 [[ 148710.104167  364319.637795 -434943.745362]]
prediction is:
 [[ 137459.92  360974.75 -442024.62]]


real location is:
 [[ 149469.145579  363568.265592 -434990.494543]]
prediction is:
 [[ 137443.69  354199.5  -433407.22]]


real location is:
 [[ 150251.758536  362816.017068 -435418.313052]]
prediction is:
 [[ 138323.56  366307.72 -428533.53]]


real location is

real location is:
 [[ 201426.489256  311239.034545 -540393.946235]]
prediction is:
 [[ 203767.64  301060.66 -530708.5 ]]


real location is:
 [[ 202081.852701  310255.20057  -588630.002922]]
prediction is:
 [[ 199226.31  296947.75 -591050.75]]


real location is:
 [[ 202670.828372  309152.786355 -649789.42726 ]]
prediction is:
 [[ 186679.33  311358.4  -601934.9 ]]


real location is:
 [[ 203185.465374  307825.633293 -728796.040477]]
prediction is:
 [[ 195620.05  296899.88 -745574.94]]


real location is:
 [[ 203499.734917  306519.696844 -793966.314149]]
prediction is:
 [[ 201289.23  263252.56 -812525.25]]


real location is:
 [[ 203678.209925  305213.008187 -841998.539988]]
prediction is:
 [[ 201289.67  256504.97 -850327.3 ]]


real location is:
 [[ 203765.601512  303955.279885 -872939.339408]]
prediction is:
 [[ 198090.06  301157.28 -840080.1 ]]


real location is:
 [[ 203782.977744  302802.7051   -899931.278234]]
prediction is:
 [[ 196661.17  295482.03 -907000.1 ]]


real location is

real location is:
 [[ 205075.845349  226533.045673 -840789.736203]]
prediction is:
 [[ 202367.16  198915.84 -837248.6 ]]


real location is:
 [[ 205178.089559  225249.123853 -839133.997064]]
prediction is:
 [[ 197735.31  211278.   -825576.9 ]]


real location is:
 [[ 205309.962926  223923.708378 -841472.927955]]
prediction is:
 [[ 197927.16  209877.77 -842623.1 ]]


real location is:
 [[ 205392.605753  222725.134897 -851742.239638]]
prediction is:
 [[ 197426.19  203913.55 -815788.44]]


real location is:
 [[ 205469.869419  221412.418853 -873643.194249]]
prediction is:
 [[ 196657.4   204653.55 -851789.9 ]]


real location is:
 [[ 205468.228774  220220.347599 -900059.011819]]
prediction is:
 [[ 194026.28  225912.42 -850099.06]]


real location is:
 [[ 205434.168142  219016.066122 -903307.726823]]
prediction is:
 [[ 198847.88  191566.66 -851568.9 ]]


real location is:
 [[ 205408.524203  217860.216064 -904698.090322]]
prediction is:
 [[ 197459.67  221161.55 -881420.44]]


real location is

real location is:
 [[ 206570.562686  137238.39067  -893334.251932]]
prediction is:
 [[ 199703.81  147507.75 -867308.56]]


real location is:
 [[ 206592.140156  135902.473927 -886346.296906]]
prediction is:
 [[ 191293.97  143404.86 -867916.9 ]]


real location is:
 [[ 206599.790949  134647.456497 -889587.533251]]
prediction is:
 [[ 199377.1   140610.69 -855513.2 ]]


real location is:
 [[ 206620.948312  133411.434966 -886370.291813]]
prediction is:
 [[ 203866.62  141564.4  -861284.5 ]]


real location is:
 [[ 206614.548004  132176.457343 -891987.535054]]
prediction is:
 [[ 202502.98  141346.52 -845936.4 ]]


real location is:
 [[ 206615.169789  130904.378674 -893771.721586]]
prediction is:
 [[ 198113.31  143191.86 -856725.6 ]]


real location is:
 [[ 206618.005517  129688.631046 -898812.868466]]
prediction is:
 [[ 204917.78  124069.85 -856737.5 ]]


real location is:
 [[ 206620.668024  128347.446932 -901114.33936 ]]
prediction is:
 [[ 202654.31  131145.47 -848742.4 ]]


real location is